In [21]:
import pandas as pd
import numpy as np
import torch
import math

In [23]:
def format_date(date_string):
    splitted = date_string.split("/")
    new_string = splitted[2]+"-"+splitted[1]+"-"+splitted[0]
    return new_string


In [24]:
def time_encoding(time):
    splitted = time.split(":")
    hour = int(splitted[0])
    minute = int(splitted[1])
    second = int(splitted[2])
    l = [hour, minute, second]
    arr = np.array(l, dtype=np.float64)

    arr[1] /= 60
    arr[2] /= 3600


    total_hours = arr[0] + arr[1] + arr[2]

    return total_hours/24

In [25]:
def features_encoding(df, feature):
    df[feature] = df[feature].astype(float)
    mean = df[feature].mean()
    sd = math.sqrt(df[feature].var())
    df[feature] = (df[feature]-mean)/sd
    return df

In [33]:
df_sleep = pd.read_csv("minute_data_007_youcef/007_longitudinal_df_dylan.csv")
df_encoded = pd.read_csv("minute_data_007_youcef/encoded.csv")

In [34]:
df_sleep["Date"] = df_sleep["Date"].apply(format_date)
df_sleep["date_only"] = df_sleep["Date"]
df_sleep["Sleep.Onset.Time"] = df_sleep["Sleep.Onset.Time"].apply(time_encoding)
df_sleep["Rise.Time"] = df_sleep["Rise.Time"].apply(time_encoding)

for i in range(3, len(df_sleep.columns)-1):
    df_sleep = features_encoding(df_sleep, df_sleep.columns[i])
    
    

In [35]:
df = pd.merge(df_encoded, df_sleep, on='date_only', how='inner')
print(df)

        Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0                     date  \
0                  0             0       39524  2024-08-20 00:00:34:000   
1                  1             1       39525  2024-08-20 00:01:34:000   
2                  2             2       39526  2024-08-20 00:02:34:000   
3                  3             3       39527  2024-08-20 00:03:34:000   
4                  4             4       39528  2024-08-20 00:04:34:000   
...              ...           ...         ...                      ...   
128823        132117        132117      171641  2025-01-10 23:55:46:000   
128824        132118        132118      171642  2025-01-10 23:56:46:000   
128825        132119        132119      171643  2025-01-10 23:57:46:000   
128826        132120        132120      171644  2025-01-10 23:58:46:000   
128827        132121        132121      171645  2025-01-10 23:59:46:000   

        Accelerometer_X  Accelerometer_Y  Accelerometer_Z  Light_Lux  \
0              0.585266    

In [36]:
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', "Date"])
df.to_csv("sleep+encoded.csv", index = False)